# Анализ вакансий DS и DA

**Описание проекта:** <br>

Исследование различий между вакансиями Data Scientist и Data Analyst.

**Цель исследования:** <br>

Показать чем различаются вакансии Data Scientist и Data Analyst.

**План работы:**

1) Загрузим данные и подготовим их к анализу.
2) Проведем исследовательский анализ данных.
3) Определение доли грейдов Junior, Junior+, Middle, Senior среди вакансий Аналитик данных и Специалист по Data Science.
4) Определение наиболее желаемых кандидатов на вакансии Аналитик данных и Специалист по Data Science по следующим параметрам: самые важные hard-skils, самые важные soft-skils. Ответ отдельно дайте для грейдов Junior, Junior+, Middle, Senior.
5) Определение типичного места работы для Аналитика данных и специалист по Data Science по следующим параметрам: ТОП-работодателей, зарплата, тип занятости, график работы. Ответ отдельно дайте для грейдов Junior, Junior+, Middle, Senior.
6) Расчет помесячной динамики количества вакансий для Аналитика данных и специалиста по Data Science. Ответ отдельно дайте для грейдов Junior, Junior+, Middle, Senior.
7) Формулирование выводов и рекомендаций.
8) Создание презентации.

**Описание данных:**

В нашем распоряжении два датасета о вакансиях представленных на сайте HH.ru c 2024-02-29 по 2024-05-07. Данные получены из API HH.ru.

`da.csv` - вакансии аналитиков данных
`ds.csv` - вакансии датасаентистов

Столбцы в двух таблицах идентичны:

- `id` -  уникальный номер вакансии
- `name` - название вакансии
- `published_at` - дата публикации вакансии(Для каждой вакансии мы сохраням только первую дату публикации)
- `alternate_url` - ссылка на вакансию
- `type` - тип вакансии
- `employer` - работодатель
- `department` - подразделение работодателя
- `area` - регион вакансии
- `experience` - требуемый опыт работы
- `key_skills` - требуемые скиллы
- `schedule` - график работы
- `employment` - тип занятости
- `description` - подробное описание вакансии
- `salary_from` - зарплата от
- `salary_to` -  зарплата до

## Загрузим данные и подготовим их к анализу

In [60]:
import pandas as pd
from pymystem3 import Mystem
import numpy as np
from tqdm.notebook import tqdm
import re

---

#### Посмотрим как выглядят данные

In [2]:
# приведем published_at сразу в формат даты
da = pd.read_csv('data/da.csv', parse_dates=['published_at'])
da.sample(5)

,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,salary_from,salary_to
9,85053195,System analyst DWH / Data Analyst,2024-05-06 09:37:32,https://hh.ru/vacancy/85053195,Открытая,Axenix (ранее Accenture),NaN,Краснодар,От 1 года до 3 лет,"['SQL', 'DWH', 'Teradata', 'Hadoop', 'Airflow'...",Удаленная работа,Полная занятость,Компания Axenix (ранее Accenture) продолжает р...,NaN,NaN
449,96401308,Аналитик данных в образовательный трек,2024-04-22 11:52:35,https://hh.ru/vacancy/96401308,Открытая,Умскул,NaN,Москва,От 1 года до 3 лет,"['Математическая статистика', 'SQL', 'BI инстр...",Удаленная работа,Полная занятость,Умскул — крупнейшая в России онлайн-школа подг...,NaN,NaN
703,97221381,Аналитик данных CRM,2024-04-17 13:10:57,https://hh.ru/vacancy/97221381,Открытая,AUXO (Атос АйТи Солюшенс энд Сервисез),NaN,Москва,От 1 года до 3 лет,"['CRM', 'MS SQL', 'Power BI']",Полный день,Полная занятость,"Обязанности: Проактивный анализ, оптимизация ...",NaN,NaN
1092,98428318,Преподаватель курса Аналитика данных (online),2024-05-06 07:55:15,https://hh.ru/vacancy/98428318,Открытая,JustCode,NaN,Алматы,От 1 года до 3 лет,"['Python', 'SQL', 'MS PowerPoint', 'Подготовка...",Удаленная работа,Частичная занятость,Чем предстоит заниматься:- подготовкой материа...,NaN,NaN
490,96523236,Аналитик данных (г. Шымкент),2024-04-09 13:38:59,https://hh.ru/vacancy/96523236,Открытая,Компания ЭВРИКА,NaN,Шымкент,От 1 года до 3 лет,"['SQL', 'Работа с большим объемом информации',...",Полный день,Полная занятость,"Требования: Знание SQL (DDL, DML) Знание Powe...",NaN,NaN


In [3]:
# приведем published_at сразу в формат даты
ds = pd.read_csv('data/ds.csv', parse_dates=['published_at'])
ds.sample(5)

,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,salary_from,salary_to
385,95703069,Team Lead Data Science,2024-03-28 14:11:32,https://hh.ru/vacancy/95703069,Открытая,СБЕР,Сбер для экспертов,Москва,Middle (3-6 years),['Юнит-экономика'],Полный день,Полная занятость,наша команда ai департамента развития корпорат...,NaN,NaN
731,98339658,Middle/Senior Data Scientist (команда СХ),2024-05-03 13:54:03,https://hh.ru/vacancy/98339658,Открытая,СБЕР,Сбер для экспертов,Москва,От 3 до 6 лет,[],Полный день,Полная занятость,Команда Клиентского опыта ПАО &quot;Сбербанк&q...,NaN,NaN
721,98275762,"Data Scientist (Deep Learning), Рекомендации и...",2024-05-02 20:24:25,https://hh.ru/vacancy/98275762,Открытая,Ozon,Ozon Информационные технологии,Москва,От 3 до 6 лет,"['IT', 'Разработка поисковых технологий', 'dat...",Удаленная работа,Полная занятость,Наша команда занимается персонализированным по...,NaN,NaN
528,96978877,Data Scientist/Analyst,2024-05-06 09:35:29,https://hh.ru/vacancy/96978877,Открытая,RapidSeedbox ltd,NaN,Санкт-Петербург,От 3 до 6 лет,"['Google Analytics', 'Data Analysis', 'Python'...",Удаленная работа,Полная занятость,Our core values are: ✔️Excellence in everythin...,1000.0,1500.0
394,95728796,Data Scientist в отдел разработки скоринговых ...,2024-03-28 18:15:23,https://hh.ru/vacancy/95728796,Открытая,Тинькофф,NaN,Москва,Middle (3-6 years),"['Machine Learning', 'Pandas', 'Python', 'ML',...",Полный день,Полная занятость,принимаем решения на основе данных и тестов. и...,NaN,NaN


---

#### Добавим флаговую переменную(которая будет принимать два значения "da" или "ds") и объеденим данные, чтобы на этапе обработке было удобнее с ними работать

In [5]:
# Создаем переменные
da['name_type'] = 'da'
ds['name_type'] = 'ds'

# объеденяем таблицы
vacancies = pd.concat((da, ds))
start_data_size = vacancies.shape[0]

---

Посмотрим общую информацию о датасете

In [6]:
vacancies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1975 entries, 0 to 781
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             1975 non-null   int64         
 1   name           1975 non-null   object        
 2   published_at   1975 non-null   datetime64[ns]
 3   alternate_url  1975 non-null   object        
 4   type           1975 non-null   object        
 5   employer       1975 non-null   object        
 6   department     614 non-null    object        
 7   area           1975 non-null   object        
 8   experience     1975 non-null   object        
 9   key_skills     1975 non-null   object        
 10  schedule       1975 non-null   object        
 11  employment     1975 non-null   object        
 12  description    1975 non-null   object        
 13  salary_from    275 non-null    float64       
 14  salary_to      197 non-null    float64       
 15  name_type      1975 non-nul

Видно что работодатели зачастую не вносят информацию о зарплате.

---

Мы хотим сравнить две когорты аналитиков и датасаентистов. В наших данных есть смежные вакансии - Аналитик данных/Data Scientist, так как непонятно к какой когорте их отнести, давайте посмотрим сколько этих данных в нашем датасете и удалим их.

In [8]:
grid = vacancies.name.str.lower().str.contains(r'data scien') \
    & (vacancies.name.str.lower().str.contains(r'analyst') \
    | vacancies.name.str.lower().str.contains(r'аналитик'))
vacancies[grid].sort_values(by='id')

,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,salary_from,salary_to,name_type
56,91765959,Дата аналитик (Data Scientist),2024-04-04 10:10:44,https://hh.ru/vacancy/91765959,Открытая,Центр финансовых технологий,NaN,Санкт-Петербург,Junior+ (1-3 years),['Pandas'],Полный день,Полная занятость,приглашаем в ml команду специалистов в области...,NaN,NaN,da
63,91765959,Дата аналитик (Data Scientist),2024-04-04 10:10:44,https://hh.ru/vacancy/91765959,Открытая,Центр финансовых технологий,NaN,Санкт-Петербург,Junior+ (1-3 years),['Pandas'],Полный день,Полная занятость,приглашаем в ml команду специалистов в области...,NaN,NaN,ds
64,91765960,Дата аналитик (Data Scientist),2024-03-28 06:25:29,https://hh.ru/vacancy/91765960,Открытая,Центр финансовых технологий,NaN,Новосибирск,Junior+ (1-3 years),['Pandas'],Полный день,Полная занятость,приглашаем в ml команду специалистов в области...,NaN,NaN,ds
57,91765960,Дата аналитик (Data Scientist),2024-03-28 06:25:29,https://hh.ru/vacancy/91765960,Открытая,Центр финансовых технологий,NaN,Новосибирск,Junior+ (1-3 years),['Pandas'],Полный день,Полная занятость,приглашаем в ml команду специалистов в области...,NaN,NaN,da
70,92068025,"Аналитик данных (data scientist, data engineer)",2024-03-12 12:42:01,https://hh.ru/vacancy/92068025,Открытая,Федеральное государственное автономное учрежде...,NaN,Москва,Middle (3-6 years),"['Документация', 'Создание моделей', 'Оформлен...",Полный день,Полная занятость,в настоящее время фгау «нии «цэпп» расширяет к...,100000.0,NaN,ds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,98551181,Аналитик данных / Data Scientist (middle+),2024-05-07 14:36:58,https://hh.ru/vacancy/98551181,Открытая,"СИБУР, Группа компаний",Сибур,Москва,От 3 до 6 лет,"['Python', 'SQL', 'Математическая статистика']",Полный день,Полная занятость,СИБУР Диджитал - это цифровой кластер в состав...,NaN,NaN,da
774,98573757,Аналитик данных/Data Scientist,2024-05-07 20:09:28,https://hh.ru/vacancy/98573757,Открытая,Консорциум Кодекс,NaN,Санкт-Петербург,От 1 года до 3 лет,"['Machine Learning', 'Data Science', 'SQL', 'P...",Полный день,Полная занятость,"Основные задачи: Разработка, тестирование и в...",NaN,NaN,ds
1171,98573757,Аналитик данных/Data Scientist,2024-05-07 20:09:28,https://hh.ru/vacancy/98573757,Открытая,Консорциум Кодекс,NaN,Санкт-Петербург,От 1 года до 3 лет,"['Machine Learning', 'Data Science', 'SQL', 'P...",Полный день,Полная занятость,"Основные задачи: Разработка, тестирование и в...",NaN,NaN,da
1181,98592181,Аналитик данных/Junior DATA SCIENTIST,2024-05-08 09:12:01,https://hh.ru/vacancy/98592181,Открытая,БиАйЭй-Технолоджиз,NaN,Москва,От 1 года до 3 лет,[],Полный день,Полная занятость,Активно формируем новую команду для нашего кру...,NaN,NaN,da


У нас 115 таких наблюдения, давайте их удалим.

In [9]:
vacancies = vacancies[~grid].sort_values(by='id')

In [16]:
print(f'Удалили {1 - vacancies.shape[0] / start_data_size:.2%} данных')

Удалили 5.82% данных


---

#### Создадим новую переменную `published_date`

In [17]:
vacancies['published_date'] = vacancies.published_at.dt.date

---

#### Посмотрим наличие дубликатов

In [18]:
n_dupl = vacancies.duplicated(
    subset='id'
).sum()
print(f'Количество дубликатов - {n_dupl}')

Количество дубликатов - 1


Посмотрим на дубликаты

In [19]:
vacancies[
    vacancies.duplicated(
        subset='id',
        keep=False
    )].sort_values(by=['name', 'employer', 'published_date'])

,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,salary_from,salary_to,name_type,published_date
434,96061431,Data Scientist,2024-04-27 10:07:00,https://hh.ru/vacancy/96061431,Открытая,СБЕР,Сбер для экспертов,Москва,Middle (3-6 years),[],Полный день,Полная занятость,корпоративно-инвестиционный блок отвечает за р...,NaN,NaN,ds,2024-04-27
419,96061431,Аналитик данных (Голос клиента),2024-04-27 10:07:00,https://hh.ru/vacancy/96061431,Открытая,СБЕР,Сбер для экспертов,Москва,От 3 до 6 лет,[],Полный день,Полная занятость,Корпоративно-инвестиционный блок отвечает за р...,NaN,NaN,da,2024-04-27


Это одна и та же вакансия, судя по всему она сначала называлась Аналитик данных, а потом ее переименовали в Data Scientist, cтоит удалить эти наблюдения потому что не понятно к какой когорте их отнести.

In [20]:
vacancies.drop_duplicates(subset='id', keep=False, inplace=True)

In [21]:
print(f'Удалили {1 - vacancies.shape[0] / start_data_size:.2%} данных')

Удалили 5.92% данных


Проверим есть ли у нас одинаковые вакансии, но с разным id

In [22]:
vacancies[
    vacancies.duplicated(
        subset=['name', 'published_date', 'employer', 'department', 'area', 'description'],
        keep=False
    )].sort_values(by=['name', 'employer', 'published_date'])

,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,salary_from,salary_to,name_type,published_date
944,97929624,Аналитик данных,2024-04-25 17:24:21,https://hh.ru/vacancy/97929624,Открытая,ГКУ Центр занятости населения города Москвы,NaN,Москва,От 1 года до 3 лет,[],Полный день,Полная занятость,Один из ключевых партнёров Центра занятости на...,150000.0,NaN,da,2024-04-25
945,97929739,Аналитик данных,2024-04-25 17:27:28,https://hh.ru/vacancy/97929739,Открытая,ГКУ Центр занятости населения города Москвы,NaN,Москва,От 1 года до 3 лет,[],Полный день,Полная занятость,Один из ключевых партнёров Центра занятости на...,NaN,NaN,da,2024-04-25


Удалим дубликат

In [23]:
vacancies.drop_duplicates(
    subset=['name', 'published_date', 'employer', 'department', 'area', 'description'],
    keep=False, inplace=True
)

In [24]:
print(f'Удалили {1 - vacancies.shape[0] / start_data_size:.2%} данных')

Удалили 6.03% данных


---

Посмотрим какие названия вакансий у нас в датасете

In [25]:
vacancies.name.value_counts()[:10]

name
Аналитик данных            195
Data Scientist             112
Data Analyst                78
Senior Data Scientist       24
Data scientist              19
Ведущий аналитик данных     16
Middle Data Scientist       15
Senior Data Analyst         13
Junior Data Analyst         12
Data analyst                11
Name: count, dtype: int64

In [27]:
vacancies.name.value_counts()[-10:]

name
Руководитель отдела Data Science                                                 1
Аналитик данных / bi-аналитик                                                    1
Middle Data Scientist (Управление Комплаенс)                                     1
Team lead Data Scientist, Маркетплейс, Контент и товары, Машинный перевод        1
Product/Data Analyst (GigaCode)                                                  1
Data Analyst (Customer Communication)                                            1
Junior Data Scientist в SberData                                                 1
Middle Data Scientist в Департамент "Занять и сберегать"                         1
Data Analyst в Департамент данных и рекомендательных систем В2С                  1
Специалист по математическому моделированию/аналитик данных/программист C++/R    1
Name: count, dtype: int64

Все просмотреть нет возможности, но тут вроде порядок.

---

Посмотрим минимальные и максимальные даты

In [38]:
print(f'Минимальная дата - {vacancies.published_at.min().strftime("%Y-%m-%d %X")}')
print(f'Максимальная дата - {vacancies.published_at.max().strftime("%Y-%m-%d %X")}')

Минимальная дата - 2024-02-29 15:01:35
Максимальная дата - 2024-05-08 13:54:28


---

Посмотрим какие регионы присутствуют в нашем датасете

In [44]:
vacancies[vacancies.name_type == 'da'].area.value_counts()[:25]

area
Москва             765
Санкт-Петербург     94
Ташкент             26
Екатеринбург        25
Алматы              23
Нижний Новгород     16
Минск               16
Новосибирск         15
Владивосток         13
Казань              12
Краснодар           12
Сербия              10
Астана               9
Ростов-на-Дону       6
Тбилиси              6
Бишкек               5
Красноярск           4
Владимир             4
Самара               4
Воронеж              4
Саратов              4
Челябинск            4
Тюмень               4
Пермь                3
Ижевск               3
Name: count, dtype: int64

---

Посмотрим значения в переменной `experience`

In [48]:
vacancies.experience.value_counts()

experience
От 1 года до 3 лет       774
От 3 до 6 лет            527
Junior+ (1-3 years)      207
Middle (3-6 years)       195
Нет опыта                 98
Более 6 лет               25
Junior (no experince)     18
Senior (6+ years)         12
Name: count, dtype: int64

Привидем значения к общему виду

In [63]:
def calc_experience(value):
    if value[0] < 'А':
        return value
    expirience = re.findall(r'\d', value)
    if not expirience:
        return 'Junior (no experince)'
    if expirience[0] == '6':
        return 'Senior (6+ years)'
    if expirience[0] == '1':
        return 'Junior+ (1-3 years)'
    if expirience[0] == '3':
        return 'Middle (3-6 years)'   

In [66]:
vacancies.experience = vacancies.experience.map(calc_experience)

---

Посмотрим значения графика работы

In [74]:
vacancies.schedule.value_counts()

schedule
Полный день         1469
Удаленная работа     312
Гибкий график         64
Сменный график        10
Вахтовый метод         1
Name: count, dtype: int64

Интересно посмотреть на вакансию с вахтовым методом:)

In [79]:
vacancies[vacancies.schedule == 'Вахтовый метод'].description.values

array(['Уважаемые соискатели, Мы в поиске сотрудника на строительный объект АГПЗ, город Свободный. Обязанности:  Актуализация базы данных по входному контролю; Выгрузка данных по запросам и проведение аналитики по полученным результатам; Формирование сводных таблиц для расчетов, расчет показателей с помощью формул.  Требования:  Высшее техническое или экономическое образование; Продвинутый пользователь MS Excel (сводные таблицы, макросы, формулы ВПР / ЕСЛИ / СУММЕСЛИ / СЧЕТЕСЛИ); Опыт работы с большими массивами данных; Желателен опыт работы в строительстве.  Условия:  Работа на строительном объекте (АГПЗ, г. Свободный, Амурская область); Официальное трудоустройство, оформление в соответствии с требованиями ТК РФ; Вахтовый метод работы 2 мес. / 1 мес.; Размещение в жилом вахтовом городке (отдельный санузел, душевая кабина в каждой комнате); Бесплатное 3-х разовое питание; Спортзал, прачечная, столовая, магазин на территории; Возможность принять участие в уникальном проекте. '],
      d

---

Посмотрим какие значения принимает переменная `employment`

In [80]:
vacancies.employment.value_counts()

employment
Полная занятость       1818
Стажировка               27
Частичная занятость       7
Проектная работа          4
Name: count, dtype: int64

---

In [90]:
vacancies.reset_index(drop=True, inplace=True)

### Создадим необходимые переменные

Проведем лемматизацию описания и результат сохраним в новый столбец `description_lemmatized`

In [115]:
def lemmatize_corpus(description: pd.Series) -> pd.Series:
    texts = ' br '.join(description.to_list())
    
    print('Запуск лематизации')
    stem = Mystem()
    text_lemm = stem.lemmatize(texts)
    
    data = []
    temp = []
    for word in tqdm(text_lemm):
        if word == 'br':
            data.append(' '.join([word for word in temp if word.isalpha()]))
            temp = []
        else:
            temp.append(word)
    data.append(' '.join([word for word in temp if word.isalpha()]))
    data = pd.Series(data, name='description_lemmatized')
    assert description.shape[0] == data.shape[0]
    return data

In [116]:
vacancies['description_lemmatized'] = lemmatize_corpus(vacancies.description)

Запуск лематизации


  0%|          | 0/970615 [00:00<?, ?it/s]

---

Создадим переменную `salary_cat`, которая будет содержать категорию заработной платы.

In [120]:
old = pd.read_excel('data/vacancies_da.xlsx')

In [122]:
old.salary_bin.value_counts()

salary_bin
ЗП не указана                285
От 200 тысяч до 300 тысяч     11
Больше 300 тысяч               7
От 100 тысяч до 200 тысяч      6
Меньше 100 тысяч               1
Name: count, dtype: int64